In [28]:
# import libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution1D, Flatten
from keras.callbacks import EarlyStopping
from keras.layers.noise import GaussianNoise

In [2]:
# read in data
datafile = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(datafile.head())

Y_train = datafile['species']
X_train = datafile.drop('species', axis=1)

# not looking at actual images for first pass, so drop id column from both test and train
X_train = X_train.drop('id', axis=1)
X_test = test.drop('id', axis=1)

# one hot encode species
lb = preprocessing.LabelBinarizer()
lb.fit(Y_train)

   id                species   margin1   margin2   margin3   margin4  \
0   1            Acer_Opalus  0.007812  0.023438  0.023438  0.003906   
1   2  Pterocarya_Stenoptera  0.005859  0.000000  0.031250  0.015625   
2   3   Quercus_Hartwissiana  0.005859  0.009766  0.019531  0.007812   
3   5        Tilia_Tomentosa  0.000000  0.003906  0.023438  0.005859   
4   6     Quercus_Variabilis  0.005859  0.003906  0.048828  0.009766   

    margin5   margin6   margin7  margin8    ...      texture55  texture56  \
0  0.011719  0.009766  0.027344      0.0    ...       0.007812   0.000000   
1  0.025391  0.001953  0.019531      0.0    ...       0.000977   0.000000   
2  0.003906  0.005859  0.068359      0.0    ...       0.154300   0.000000   
3  0.021484  0.019531  0.023438      0.0    ...       0.000000   0.000977   
4  0.013672  0.015625  0.005859      0.0    ...       0.096680   0.000000   

   texture57  texture58  texture59  texture60  texture61  texture62  \
0   0.002930   0.002930   0.03515

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [34]:
model = Sequential([
    Dropout(0.3, input_shape=(X_train.shape[1],)),
    Dense(8000),
    Activation('relu'),
    Dropout(0.5),
    Dense(100),
    Activation('relu'),
    Dropout(0.2),
    Dense(lb.classes_.size),
    Activation('softmax'),
])

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    X_train.as_matrix(), 
    lb.transform(Y_train), 
    validation_split=0.1,
    nb_epoch=100
)

Train on 891 samples, validate on 99 samples
Epoch 1/100
891/891 [==============================] - 1s - loss: 4.5661 - acc: 0.0348 - val_loss: 4.5056 - val_acc: 0.0303
Epoch 2/100
891/891 [==============================] - 1s - loss: 4.3463 - acc: 0.0774 - val_loss: 4.2057 - val_acc: 0.0707
Epoch 3/100
891/891 [==============================] - 1s - loss: 3.9274 - acc: 0.1111 - val_loss: 3.7654 - val_acc: 0.1010
Epoch 4/100
891/891 [==============================] - 1s - loss: 3.4521 - acc: 0.1627 - val_loss: 3.2398 - val_acc: 0.1818
Epoch 5/100
891/891 [==============================] - 1s - loss: 3.0256 - acc: 0.2402 - val_loss: 2.7758 - val_acc: 0.3030
Epoch 6/100
891/891 [==============================] - 1s - loss: 2.6743 - acc: 0.2963 - val_loss: 2.4412 - val_acc: 0.4343
Epoch 7/100
891/891 [==============================] - 1s - loss: 2.4022 - acc: 0.3434 - val_loss: 2.0807 - val_acc: 0.5051
Epoch 8/100
891/891 [==============================] - 1s - loss: 2.2508 - acc: 0.3625 

In [35]:
predictions = model.predict_proba(X_test.as_matrix())

594/594 [==============================] - 0s     


In [36]:
labels = test['id']
headers = lb.classes_.tolist()
headers = ['id']+headers
results = pd.DataFrame(data=predictions)
labels = pd.DataFrame(labels)
results = pd.concat([labels, results], axis=1)
results.columns = headers
results.to_csv('results.csv', index=False)